In [539]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import svm

In [522]:
customers=pd.read_csv('dataset_num.csv')
y=pd.read_csv('Fraud.csv')

In [437]:
customers.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy,ClaimSize,Age_na,DriverRating_na
0,3,5,7,7,2,7,6,1,1,3,...,1,1,4,1,3,1994,3,55526.069421,False,False
1,5,3,7,7,2,3,6,4,2,3,...,1,1,4,4,1,1994,2,59294.455571,False,False
2,11,5,1,7,2,6,11,2,2,2,...,1,1,4,4,1,1994,2,71756.941320,False,False
3,7,2,3,18,1,2,7,1,2,2,...,1,1,3,4,1,1994,3,7584.153493,False,False
4,4,1,3,7,2,3,5,3,2,2,...,1,1,1,4,1,1994,2,98545.374339,False,False


In [542]:
customers['Days_Policy_Claim'].value_counts()

3    11503
1       47
2       13
4        1
Name: Days_Policy_Claim, dtype: int64

In [470]:
customers.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'PolicyNumber', 'RepNumber', 'Deductible',
       'DriverRating', 'Days_Policy_Accident', 'Days_Policy_Claim',
       'PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder',
       'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year',
       'BasePolicy', 'ClaimSize', 'Age_na', 'DriverRating_na'],
      dtype='object')

In [440]:
from sklearn.model_selection import StratifiedKFold

In [523]:
#del customers['Month']
del customers['WeekOfMonth']
del customers['WeekOfMonthClaimed']
#del customers['MonthClaimed']
#del customers['DayOfWeek']
#del customers['DayOfWeekClaimed']
del customers['PolicyNumber']
del customers['Year']
del customers['Age_na']
del customers['DriverRating_na']

In [524]:
customers['FraudFound_P']=y
customers.dropna(inplace=True)
Fraud=customers[customers['FraudFound_P']==1]
NotFraud=customers[customers['FraudFound_P']==0]
Y_fraud=Fraud['FraudFound_P'].values
Y_NotFraud=NotFraud['FraudFound_P'].values
del Fraud['FraudFound_P']
del NotFraud["FraudFound_P"]

In [477]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [525]:
Y_NotFraud.shape[0]* 0.03

326.37

In [526]:
X_train_fraud,X_test_fraud,Y_train_fraud,Y_test_fraud=train_test_split(Fraud.values,Y_fraud,test_size=0.5)

In [389]:
Y_test_fraud.shape

(343,)

In [533]:
X_train_Notfraud,X_test_Notfraud,Y_train_Notfraud,Y_test_Notfraud=train_test_split(NotFraud.values,Y_NotFraud,test_size=0.90)

In [455]:
X_train_Notfraud.shape[0]

543

In [392]:
X_train_fraud.shape

(342, 25)

In [78]:
## Joining the X_train Y_train X_test Y_test

In [534]:
X_train=np.concatenate([X_train_fraud,X_train_Notfraud])
Y_train=np.concatenate([Y_train_fraud,Y_train_Notfraud])
X_test=np.concatenate([X_test_fraud,X_test_Notfraud])
Y_test=np.concatenate([Y_test_fraud,Y_test_Notfraud])

In [535]:
df_new=pd.DataFrame(X_train)
df_new['FraudFound_P']=Y_train
df_new.columns=customers.columns
df_new = df_new.sample(frac=1).reset_index(drop=True)
Y_train=df_new['FraudFound_P'].values
del df_new['FraudFound_P']
X_train=df_new.values

In [536]:
clf=RandomForestClassifier(n_estimators=40,max_features=0.5)
clf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [537]:
Y_pred=clf.predict(X_test)
print(accuracy_score(Y_test,Y_pred))
print(confusion_matrix(Y_pred,Y_test))
print(f1_score(Y_test,Y_pred))

0.9515540207202763
[[9642  341]
 [ 150    2]]
0.00808080808080808


In [532]:
Y_pred=clf.predict(X_train)
print(accuracy_score(Y_train,Y_pred))
print(confusion_matrix(Y_pred,Y_train))
print(f1_score(Y_train,Y_pred))

0.9984794728839331
[[1631    3]
 [   0  339]]
0.9955947136563876


In [99]:
np.save('X_train.npy',X_train)
np.save('X_test.npy',X_test)
np.save('Y_train.npy',Y_train)
np.save('Y_test.npy',Y_test)

In [42]:
X=customers[customers['AgeOfVehicle']==3]['FraudFound_P'].values
Fraud=len(np.where(X==1)[0])
NotFraud=len(np.where(X==0)[0])
TotalClaims=len(X)
print('Fraud',Fraud)
print('Not Fraud',NotFraud)
print('Total Claims',TotalClaims)
## Percentage of fraud claim
print('Fraud',(Fraud/TotalClaims)*100,'%')
print('Not Fraud',(NotFraud/TotalClaims)*100,'%')

Fraud 6
Not Fraud 170
Total Claims 176
Fraud 3.4090909090909087 %
Not Fraud 96.5909090909091 %
